In [ ]:
# Generate a Gaussian Random Field in a 3D box

In [1]:
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
from numpy import inf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab
from random import gauss
from random import seed
import pandas as pd
from pandas import Series
from pandas.plotting import autocorrelation_plot
import scipy
from scipy import signal

np.set_printoptions(threshold=5)

Matplotlib is building the font cache; this may take a moment.


In [2]:
from google.colab import drive 
import os

drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/Preheating/')

Mounted at /content/drive/


# 1D Case



Let us start from the simple 1D case.

We define the box in real space as a collection of $N$ points $n \in (0, \dots, N-1)$.

The lattice spacing is given by $\delta x = L/N$, where $L$ is the physical length of the box.

We impose periodic boundary conditions: given a function $f(n)$ we have that $f(n + N) = f(n)$.

The periodic boundary conditions impose a discretization of momenta, that we denote by $\tilde{n}$:

$$
\tilde{n} = -\frac{N}{2} + 1, -\frac{N}{2} + 2, \dots, -1, 0, 1, \dots, \frac{N}{2} - 1, \frac{N}{2} \,.
$$

We define the discrete Fourier transform of the function $\phi$ as follows

$$
\phi(n) = \frac{1}{N} \sum_{\tilde{n}=0}^{N-1} \tilde{\phi}(\tilde{n}) e^{-i \frac{2 \pi}{N} n \tilde{n}} \,.
$$

The inverse Fourier transform will be

$$
\tilde\phi(\tilde{n}) = \sum_{n=0}^{N-1} \phi(n) e^{i \frac{2 \pi}{N} n \tilde{n}} \,.
$$

Note that one can recover the momentum in the continuum as

$$
k(\tilde{n}) = \frac{2 \pi \tilde{n}}{L} \,.
$$

Since we have a real field, we need to impose the straightforward condition

$$
\tilde{\phi}(-\tilde{n}) = \tilde{\phi}^*(\tilde{n}) \,.
$$

Let's implement these conditions. Note that in this section we will not pay attention to the size of the perturbations: we will only focus on understanding how to generate the fluctuations of a real scalar field starting from the Fourier space and checking carefully that the imaginary part is negligible. We will take care of the size of the fluctuations in the 3D section below.

First, define the number of points in the box $N$ and the physical length $L$.

In [ ]:
q = 10**5     # q = mp/m
N = 100       # # no. of points
L = 100*q     # physical length, in geometric units

Var = (N)/(q**2)
m = 1/q
H = 1/q

print("The physical length is: %d." %L)
print("The number of points in the box is: %d." %N)

The physical length is: 10000000.
The number of points in the box is: 100.


Then we generate a 1D grid in Fourier space as explained in the paragraph above.

In [ ]:
nPos = np.array(np.arange(1, N/2+1)) # NB: need (N+1)/2 because upper extreme not included in np.arange
nNeg = np.array(np.arange(-N/2+1, 0)) # NB: need -N/2+1 because point -N/2 needs to be excluded
n1D = np.asarray(np.concatenate((nNeg, 0.0, nPos), axis = None))

np.set_printoptions(threshold=np.inf)
print("This should be of the form [-N/2+1, -N/2+2, ..., -1, 0, 1, ..., N/2-1, N/2]:")
print(n1D)
np.set_printoptions(threshold=1000)

This should be of the form [-N/2+1, -N/2+2, ..., -1, 0, 1, ..., N/2-1, N/2]:
[-49. -48. -47. -46. -45. -44. -43. -42. -41. -40. -39. -38. -37. -36.
 -35. -34. -33. -32. -31. -30. -29. -28. -27. -26. -25. -24. -23. -22.
 -21. -20. -19. -18. -17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -3.  -2.  -1.   0.   1.   2.   3.   4.   5.   6.
   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.
  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.
  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.  47.  48.
  49.  50.]


Generate wavenumbers to compute the frequency of each mode. As said, we do not pay attention to the size of the fluctuations, therefore we set m = H = Var = 1.

In [ ]:
# Set the mass, H0 a dn the variance Var to 1.

# Define k as reported in the paragraph above.

k = np.array(2 * np.pi / L * n1D) # NB: need (N+1)/2 because upper extreme not included in arange
kSquared = k**2

# Print k.

np.set_printoptions(threshold=np.inf)
print("k should be of the form [2pi/L (-N/2 + 1), 2pi/L (-N/2), ..., -2pi/L, 0, 2pi/L, ..., 2pi/L (N/2-1), 2pi/L (N/2)]:")
print(k) # It should give -N/2+1, -N/2+2, ..., -1, 0, 1, ..., N/2-1, N/2
np.set_printoptions(threshold=1000)

# Define the energy of the mode k.
omega = np.sqrt(k**2 + m**2)

k should be of the form [2pi/L (-N/2 + 1), 2pi/L (-N/2), ..., -2pi/L, 0, 2pi/L, ..., 2pi/L (N/2-1), 2pi/L (N/2)]:
[-3.07876080e-05 -3.01592895e-05 -2.95309709e-05 -2.89026524e-05
 -2.82743339e-05 -2.76460154e-05 -2.70176968e-05 -2.63893783e-05
 -2.57610598e-05 -2.51327412e-05 -2.45044227e-05 -2.38761042e-05
 -2.32477856e-05 -2.26194671e-05 -2.19911486e-05 -2.13628300e-05
 -2.07345115e-05 -2.01061930e-05 -1.94778745e-05 -1.88495559e-05
 -1.82212374e-05 -1.75929189e-05 -1.69646003e-05 -1.63362818e-05
 -1.57079633e-05 -1.50796447e-05 -1.44513262e-05 -1.38230077e-05
 -1.31946891e-05 -1.25663706e-05 -1.19380521e-05 -1.13097336e-05
 -1.06814150e-05 -1.00530965e-05 -9.42477796e-06 -8.79645943e-06
 -8.16814090e-06 -7.53982237e-06 -6.91150384e-06 -6.28318531e-06
 -5.65486678e-06 -5.02654825e-06 -4.39822972e-06 -3.76991118e-06
 -3.14159265e-06 -2.51327412e-06 -1.88495559e-06 -1.25663706e-06
 -6.28318531e-07  0.00000000e+00  6.28318531e-07  1.25663706e-06
  1.88495559e-06  2.51327412e-06  3.14159

Generate the random phases for the left and right modes, taken from a flat distribution in (0, 2pi):

In [ ]:
# Generate a left and right phases, with angles taken from a flat distribution in [0, 2pi).

ThetaL = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N)
ThetaR = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N)

PhaseL = np.exp(ThetaL * 1j)
PhaseR = np.exp(ThetaR * 1j)

Generate the amplitude of the fluctuation both for the left and right modes, using a Raileigh distribution with variance Var (recall that in the 1D case we do not pay attention to the size of the fluctuations).

In [ ]:
# Generate scalar field fluctuations from a Raileigh distribution with variance Var.

DeltaPhiL = np.random.rayleigh(scale = Var**0.5, size = N)
DeltaPhiR = np.random.rayleigh(scale = Var**0.5, size = N)

Define the unconstrained field and field derivatives as in Eq. 437-438 of 2006.15122. Note that if we compute the inverse Fourier transform of this field would not get a real field.

In [ ]:
# Define the unconstrained field and field derivative.

DeltaPhiUNC = 1./np.sqrt(2) * (np.abs(DeltaPhiL) * PhaseL + np.abs(DeltaPhiR) * PhaseR)
DeltaPhiPrimeUNC = 1j*omega/np.sqrt(2) * (np.abs(DeltaPhiL) * PhaseL - np.abs(DeltaPhiR) * PhaseR) - H0*DeltaPhiUNC

Build the constrained field and field derivatives, recalling that:
1. In Python the Fourier components of $\phi$ are arranged in an array as follows: $[\tilde\phi(0), \tilde\phi(1), \tilde\phi(2), ..., \tilde\phi(N/2), \tilde\phi(-N/2+1), \tilde\phi(-N/2+2), ..., \tilde\phi(-1)]$.
2. We need to impose that $\tilde\phi(-\tilde{n}) = \tilde\phi^*(\tilde{n})$.

In [ ]:
# Build the constrained field and field derivative fluctuations.

DeltaPhi0Mode = 1./2.*(DeltaPhiUNC[int(N/2-1)]+np.conjugate(DeltaPhiUNC[int(N/2-1)]))
DeltaPhiK = np.concatenate((DeltaPhiUNC[int(N/2):],DeltaPhiUNC[0:int(N/2-1)]))
DeltaPhiKConstrained = 1./2.*(DeltaPhiK + np.flip(np.conjugate(DeltaPhiK)))
DeltaPhi = np.concatenate((DeltaPhi0Mode.reshape(1,),DeltaPhiKConstrained))

DeltaPhiPrime0Mode = 1./2.*(DeltaPhiPrimeUNC[int(N/2-1)]+np.conjugate(DeltaPhiPrimeUNC[int(N/2-1)]))
DeltaPhiPrimeK = np.concatenate((DeltaPhiPrimeUNC[int(N/2):],DeltaPhiPrimeUNC[0:int(N/2-1)]))
DeltaPhiPrimeKConstrained = 1./2.*(DeltaPhiPrimeK + np.flip(np.conjugate(DeltaPhiPrimeK)))
DeltaPhiPrime = np.concatenate((DeltaPhiPrime0Mode.reshape(1,),DeltaPhiPrimeKConstrained))

np.set_printoptions(threshold=np.inf)
print("Check that the constrained field satisfies the required symmetry:")
print(DeltaPhi)
np.set_printoptions(threshold=1000)

Check that the constrained field satisfies the required symmetry:
[-1.51246082e-05+0.00000000e+00j  1.35756779e-04-7.28207415e-05j
  1.71832242e-05-3.43549408e-05j -1.57483878e-04+4.69253745e-05j
  7.63713348e-05-3.23977854e-05j  7.64499448e-05+7.90631849e-05j
 -4.88570180e-05+1.17366177e-05j  3.57554865e-05+1.53528072e-05j
  8.24212641e-07-1.24456840e-04j  1.27138950e-05+4.40616096e-05j
 -1.88582900e-05-4.42358283e-05j -6.21938778e-05+6.09624668e-05j
 -2.00795985e-05+3.08853777e-05j  2.10429431e-05-9.41623937e-05j
  8.66246578e-05+2.99837732e-05j -8.46281516e-05-3.76784493e-05j
  1.17710288e-04-3.98060237e-05j -7.80932708e-05-5.66256521e-05j
  7.13637562e-05+8.53559098e-06j  2.14131380e-05+2.94109289e-05j
  1.67733033e-05-1.28621658e-05j -8.98190896e-06-7.17825183e-05j
  5.96552886e-06+5.35656457e-06j -1.53394573e-04+4.13999848e-05j
  4.01124879e-05+6.57807437e-05j -1.31392068e-05+5.67913860e-05j
  2.30495848e-06-4.05971928e-05j -8.71629942e-05+8.71920556e-05j
  3.63541875e-05+4.11111

Now we can finally compute the inverse Fourier transform, that should give a real result.

In [ ]:
# Compute the inverse Fourier transform, that should give a real result.

DeltaPhiReal = np.fft.ifft(DeltaPhi)
DeltaPhiPrimeReal = np.fft.ifft(DeltaPhiPrime)

ImPhi=np.max(np.imag(DeltaPhiReal))
ImPhiPrime=np.max(np.imag(DeltaPhiPrimeReal))

np.set_printoptions(threshold=np.inf)
print("Check that the inverse Fourier transform gives a real field:")
print(DeltaPhiReal)
print("In particular the maximum imaginary component of field and field derivative are %.20f, %.20f respectively." %(ImPhi, ImPhiPrime))
np.set_printoptions(threshold=1000)

Check that the inverse Fourier transform gives a real field:
[ 4.61758257e-06+2.71050543e-22j -2.60063061e-06-6.09863722e-22j
  9.15233537e-06+2.71050543e-22j  5.68989988e-06+0.00000000e+00j
 -5.91107421e-06-1.35525272e-22j -1.35863327e-06-3.72694497e-22j
  2.27979976e-06+5.42101086e-22j  1.22781526e-05+2.71050543e-22j
 -5.26820043e-06+1.35525272e-22j -1.13593153e-05-5.42101086e-22j
  1.04220853e-05-1.08420217e-21j -3.89783764e-06-1.35525272e-22j
 -5.33496274e-07+5.42101086e-22j  1.03225437e-05-2.71050543e-22j
  2.05718405e-06+9.82558219e-22j  1.46406662e-05-1.08420217e-21j
  1.69723131e-05-1.08420217e-21j -3.44086047e-07-4.06575815e-22j
  5.81745702e-06+0.00000000e+00j -8.43273852e-06-6.77626358e-23j
  1.69382085e-05-2.19284496e-22j  1.12312646e-05-1.12697324e-22j
 -1.74305551e-05-2.19284496e-22j  1.28856455e-05+1.78124629e-22j
  8.86032808e-06-1.82775651e-22j  3.81371056e-07+6.25369138e-23j
  1.02079864e-05-2.26341045e-22j -8.41065336e-06+9.02371715e-22j
 -4.02288472e-07-7.23527340e-

In [ ]:
ndim = 1
Coords = (np.indices((N,)*ndim)).reshape(ndim, -1).T
ReshapedFR = DeltaPhiReal.reshape(N,1)
FR = np.concatenate((np.real(Coords),np.real(ReshapedFR)),axis=1)
FRDF = pd.DataFrame(FR, columns=('x','field'))
print(FRDF.head())

ReshapedDFR = DeltaPhiPrimeReal.reshape(N,1)
DFR = np.concatenate((np.real(Coords),np.real(ReshapedDFR)),axis=1)
DFRDF = pd.DataFrame(DFR, columns=('x','field'))
print(DFRDF.head())

     x     field
0  0.0  0.000005
1  1.0 -0.000003
2  2.0  0.000009
3  3.0  0.000006
4  4.0 -0.000006
     x     field
0  0.0 -0.000005
1  1.0  0.000003
2  2.0 -0.000009
3  3.0 -0.000006
4  4.0  0.000006


In [ ]:
FRDF.to_csv('FieldRealization.csv', header=None, index=None)
DFRDF.to_csv('DerivativeFieldRealization.csv', header=None, index=None)

## 2D case

In 2D the same equations as in the 1D section apply, except that $n$ and $\tilde{n}$ become two component vectors $\vec{n} = (n_1, n_2)$ and $\tilde{\vec{n}} = (\tilde{\vec{n}}_1, \tilde{\vec{n}_2})$, so that we can define a lattice both in real and in Fourier space.

The field $\phi$ and the Fourier transform of the field $\tilde\phi$ can be represented as matrices with one value for each point of the grid.

Schematically, the Fourier transform takes the form

$$
\begin{pmatrix}
\tilde{\phi}_{0,0} & \tilde{\phi}_{0,1} & \tilde{\phi}_{0,2} & \dots &\tilde{\phi}_{0,N-1} \\
\tilde{\phi}_{1,0} & \tilde{\phi}_{1,1} & \tilde{\phi}_{1,2} & \dots &\tilde{\phi}_{1,N} \\
\dots & \dots & \dots & \dots & \dots \\
\tilde{\phi}_{N-1,0} & \tilde{\phi}_{N-1,1} & \dots & \dots & \tilde\phi_{N-1,N-1}
\end{pmatrix}
$$

In order to get a real field in real space, we need to impose a few conditions on this matrix:

$$
\tilde{\phi}_{0, a} = \tilde\phi_{0,N-a}^* \,, \\
\tilde{\phi}_{a, 0} = \tilde\phi^*_{N-a, 0} \,, \\
\tilde{\phi}_{a, b} = \tilde{\phi}^*_{N-a, N-b} \,,
$$

as we can see more clearly with the following example. In fact, let us define a real array:

In [ ]:
a = np.random.rand(4,4)

Compute the 2D discrete Fourier transform of a:

In [ ]:
np.fft.fft2(a)

array([[ 6.42856665+0.j        , -0.26839899-0.57080028j,
         0.28900257+0.j        , -0.26839899+0.57080028j],
       [-2.11638823+0.5994741j , -1.05958198+0.01643976j,
         1.13982717-0.54310531j, -1.21419547+0.36647415j],
       [ 1.07204699+0.j        ,  0.41546865+0.59121427j,
        -0.70670158+0.j        ,  0.41546865-0.59121427j],
       [-2.11638823-0.5994741j , -1.21419547-0.36647415j,
         1.13982717+0.54310531j, -1.05958198-0.01643976j]])

where we can clearly see all the symmetries listed above. Then we proceed with the building of an object that, if inversely Fourier transformed, gives a real scalar.

In [ ]:
# Define the number of points on the grid, variance, mass

N = 6
Var = 1

In [ ]:
# Generate 00-component of Phi

Phi00L = np.random.rayleigh(scale = Var**0.5, size = 1)
Phi00R = np.random.rayleigh(scale = Var**0.5, size = 1)

Phi00 = 1./np.sqrt(2.)*(np.abs(Phi00L) + np.abs(Phi00R))

In [ ]:
# Generate 0a-components of Phi

Phi0aL = np.random.rayleigh(scale = Var**0.5, size = N-1)
Phi0aR = np.random.rayleigh(scale = Var**0.5, size = N-1)
Theta0aL = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Theta0aR = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Phase0aL = np.exp(1j*Theta0aL)
Phase0aR = np.exp(1j*Theta0aR)

Phi0aTemp = 1./np.sqrt(2.)*(np.abs(Phi0aL)*Phase0aL + np.abs(Phi0aR)*Phase0aR)

Phi0a = 1./2. * (Phi0aTemp + np.conjugate(np.flip(Phi0aTemp)))

In [ ]:
# Generate a0-components of Phi

Phia0L = np.random.rayleigh(scale = Var**0.5, size = N-1)
Phia0R = np.random.rayleigh(scale = Var**0.5, size = N-1)
Thetaa0L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Thetaa0R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Phasea0L = np.exp(1j*Thetaa0L)
Phasea0R = np.exp(1j*Thetaa0R)

Phia0Temp = 1./np.sqrt(2.)*(np.abs(Phia0L)*Phasea0L + np.abs(Phia0R)*Phasea0R)

Phia0 = 1./2. * (Phia0Temp + np.conjugate(np.flip(Phia0Temp)))

In [ ]:
# Generate ab-components of Phi

PhiabL = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
PhiabR = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
ThetaabL = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
ThetaabR = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
PhaseabL = np.exp(1j*ThetaabL)
PhaseabR = np.exp(1j*ThetaabR)

PhiabTemp = 1./np.sqrt(2.)*(np.abs(PhiabL)*PhaseabL + np.abs(PhiabR)*PhaseabR)

PhiabTempTemp = np.flip(np.conjugate(np.flip(PhiabTemp, axis=1)), axis=0)

Phiab = 1./2. * (PhiabTemp + PhiabTempTemp)

In [ ]:
# Finally put everything together

Phi0i = np.concatenate((Phi00, Phi0a))
Phiai = np.concatenate((Phia0.reshape(N-1,1), Phiab), axis = 1)
Phiij = np.concatenate((Phi0i.reshape(1,N), Phiai))

In [ ]:
# The final 2D box is given by

Phiij

array([[ 1.94274504+0.j        ,  0.76628832-0.10586201j,
         0.90945871-0.04508045j, -1.22475473+0.j        ,
         0.90945871+0.04508045j,  0.76628832+0.10586201j],
       [-0.48407955-0.66040154j,  0.55215734+0.41095867j,
         0.17482299-1.03328279j, -0.16287288+0.59263666j,
        -0.16291798-0.00374351j,  0.25428533-0.08793064j],
       [ 0.69768832-0.73910913j,  1.41927129-0.38350793j,
         0.44133939+1.15829939j, -0.10393361+1.47606765j,
         0.83632637+0.88959465j, -0.26692188-0.5044812j ],
       [ 0.56689845+0.j        ,  1.44118783-0.60447201j,
        -0.21993727+0.24891955j, -0.40496901+0.j        ,
        -0.21993727-0.24891955j,  1.44118783+0.60447201j],
       [ 0.69768832+0.73910913j, -0.26692188+0.5044812j ,
         0.83632637-0.88959465j, -0.10393361-1.47606765j,
         0.44133939-1.15829939j,  1.41927129+0.38350793j],
       [-0.48407955+0.66040154j,  0.25428533+0.08793064j,
        -0.16291798+0.00374351j, -0.16287288-0.59263666j,
         

In [ ]:
# Check that the scalar field is real.

np.fft.ifft2(Phiij)

array([[ 0.3628957 +0.j,  0.23357831+0.j, -0.17166332+0.j,
         0.02016238+0.j, -0.12677202+0.j,  0.17127579+0.j],
       [-0.09218698+0.j,  0.12924087+0.j, -0.19423957+0.j,
         0.19553471+0.j,  0.13278291+0.j,  0.26521844+0.j],
       [ 0.22550742+0.j,  0.03316699+0.j,  0.14977672+0.j,
         0.19629369+0.j, -0.18137254+0.j, -0.06342076+0.j],
       [ 0.1991612 +0.j, -0.00642527+0.j,  0.06949069+0.j,
         0.27471079+0.j, -0.01552895+0.j,  0.10182194+0.j],
       [-0.03224354+0.j,  0.27780444+0.j, -0.0142537 +0.j,
        -0.12450126+0.j,  0.00810522+0.j,  0.29048222+0.j],
       [ 0.0151136 +0.j, -0.11973711+0.j,  0.01880919+0.j,
         0.01343977+0.j,  0.00561314+0.j, -0.30489607+0.j]])

## 3D case

We want to generate a box containing the quantum fluctuations of a real scalar field as explained in Sec. 7.1 of 2006.15122.

The basic equations are Eq. 436-437, in which the amplitude of the left and right modes is taken from a Raileigh distribution with variance given in Eq. 436. The phases are randomly chosen using a flat distribution in (0, 2 \pi).

The results for \tilde\phi and \phi in this section are dimensionless, hence for us they are expressed in units of Mp (in the notation of Figueroa et al., f = Mp).

In [2]:
# Define relevant parameters

q = 10**5 # q = mp/m
N = 100
L = 100*q

Var = (N**3)/(q**2)
m = 1/q
H = 1/q

In [3]:
# Define the array of wavenumbers

k1 = np.array(2 * np.pi / L * np.arange(0, (N+1)/2)) # NB: need (N+1)/2 because upper extreme not included in arange
k2 = np.array(2 * np.pi / L * np.arange(-N/2+1, 0)) # NB: need -N/2+1 because point -N/2 needs to be excluded
ktemp = np.asarray(np.concatenate((k1, k2), axis = None))

a = np.meshgrid(ktemp,ktemp,ktemp)[0]
b = np.meshgrid(ktemp,ktemp,ktemp)[1]
c = np.meshgrid(ktemp,ktemp,ktemp)[2]

k0 = np.concatenate((b.reshape(N,N,N,1), a.reshape(N,N,N,1), c.reshape(N,N,N,1)), axis=3)

k0Squared = k0[:,:,:,0]**2 + k0[:,:,:,1]**2 + k0[:,:,:,2]**2

omega = np.sqrt(k0Squared + m**2)

In [4]:
# Generate 00-component of Phi and PhiPrime

Phi_000_L = np.random.rayleigh(scale = Var**0.5, size = 1)
Phi_000_R = np.random.rayleigh(scale = Var**0.5, size = 1)

Phi_000 = 1./np.sqrt(2.)*(np.abs(Phi_000_L) + np.abs(Phi_000_R))

PhiPrime_000 = omega[0,0,0]/np.sqrt(2.)*(np.abs(Phi_000_L) - np.abs(Phi_000_R)) - H * Phi_000

In [5]:
# Generate a00-components of Phi and PhiPrime

Phi_a00_L = np.random.rayleigh(scale = Var**0.5, size = N-1)
Phi_a00_R = np.random.rayleigh(scale = Var**0.5, size = N-1)
Theta_a00_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Theta_a00_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Phase_a00_L = np.exp(1j*Theta_a00_L)
Phase_a00_R = np.exp(1j*Theta_a00_R)

Phi_a00_Temp = 1./np.sqrt(2.)*(np.abs(Phi_a00_L)*Phase_a00_L + np.abs(Phi_a00_R)*Phase_a00_R)
Phi_a00 = 1./2. * (Phi_a00_Temp + np.conjugate(np.flip(Phi_a00_Temp)))

PhiPrime_a00_Temp = omega[0:N-1,0,0]/np.sqrt(2.)*(np.abs(Phi_a00_L)*Phase_a00_L - np.abs(Phi_a00_R)*Phase_a00_R) - H * Phi_a00
PhiPrime_a00 = 1./2. * (PhiPrime_a00_Temp + np.conjugate(np.flip(PhiPrime_a00_Temp)))

In [6]:
# Generate 0b0-components of Phi and PhiPrime

Phi_0b0_L = np.random.rayleigh(scale = Var**0.5, size = N-1)
Phi_0b0_R = np.random.rayleigh(scale = Var**0.5, size = N-1)
Theta_0b0_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Theta_0b0_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Phase_0b0_L = np.exp(1j*Theta_0b0_L)
Phase_0b0_R = np.exp(1j*Theta_0b0_R)

Phi_0b0_Temp = 1./np.sqrt(2.)*(np.abs(Phi_0b0_L)*Phase_0b0_L + np.abs(Phi_0b0_R)*Phase_0b0_R)
Phi_0b0 = 1./2. * (Phi_0b0_Temp + np.conjugate(np.flip(Phi_0b0_Temp)))

PhiPrime_0b0_Temp = omega[0,0:N-1,0]/np.sqrt(2.)*(np.abs(Phi_0b0_L)*Phase_0b0_L - np.abs(Phi_0b0_R)*Phase_0b0_R) - H * Phi_0b0
PhiPrime_0b0 = 1./2. * (PhiPrime_0b0_Temp + np.conjugate(np.flip(PhiPrime_0b0_Temp)))

In [7]:
# Generate 00c-components of Phi and PhiPrime

Phi_00c_L = np.random.rayleigh(scale = Var**0.5, size = N-1)
Phi_00c_R = np.random.rayleigh(scale = Var**0.5, size = N-1)
Theta_00c_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Theta_00c_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = N-1)
Phase_00c_L = np.exp(1j*Theta_00c_L)
Phase_00c_R = np.exp(1j*Theta_00c_R)

Phi_00c_Temp = 1./np.sqrt(2.)*(np.abs(Phi_00c_L)*Phase_00c_L + np.abs(Phi_00c_R)*Phase_00c_R)
Phi_00c = 1./2. * (Phi_00c_Temp + np.conjugate(np.flip(Phi_00c_Temp)))

PhiPrime_00c_Temp = omega[0,0,0:N-1]/np.sqrt(2.)*(np.abs(Phi_00c_L)*Phase_00c_L - np.abs(Phi_00c_R)*Phase_00c_R) - H * Phi_00c
PhiPrime_00c = 1./2. * (PhiPrime_00c_Temp + np.conjugate(np.flip(PhiPrime_00c_Temp)))

In [8]:
# Generate 0bc-components of Phi

Phi_0bc_L = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Phi_0bc_R = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Theta_0bc_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Theta_0bc_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Phase_0bc_L = np.exp(1j*Theta_0bc_L)
Phase_0bc_R = np.exp(1j*Theta_0bc_R)

Phi_0bc_Temp = 1./np.sqrt(2.)*(np.abs(Phi_0bc_L)*Phase_0bc_L + np.abs(Phi_0bc_R)*Phase_0bc_R)
Phi_0bc_TempTemp = np.flip(np.conjugate(np.flip(Phi_0bc_Temp, axis=1)), axis=0)
Phi_0bc = 1./2. * (Phi_0bc_Temp + Phi_0bc_TempTemp)

PhiPrime_0bc_Temp = omega[0,0:N-1,0:N-1]/np.sqrt(2.)*(np.abs(Phi_0bc_L)*Phase_0bc_L - np.abs(Phi_0bc_R)*Phase_0bc_R) - H * Phi_0bc
PhiPrime_0bc_TempTemp = np.flip(np.conjugate(np.flip(PhiPrime_0bc_Temp, axis=1)), axis=0)
PhiPrime_0bc = 1./2. * (PhiPrime_0bc_Temp + PhiPrime_0bc_TempTemp)

In [9]:
# Generate a0c-components of Phi

Phi_a0c_L = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Phi_a0c_R = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Theta_a0c_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Theta_a0c_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Phase_a0c_L = np.exp(1j*Theta_a0c_L)
Phase_a0c_R = np.exp(1j*Theta_a0c_R)

Phi_a0c_Temp = 1./np.sqrt(2.)*(np.abs(Phi_a0c_L)*Phase_a0c_L + np.abs(Phi_a0c_R)*Phase_a0c_R)
Phi_a0c_TempTemp = np.flip(np.conjugate(np.flip(Phi_a0c_Temp, axis=1)), axis=0)
Phi_a0c = 1./2. * (Phi_a0c_Temp + Phi_a0c_TempTemp)

PhiPrime_a0c_Temp = omega[0:N-1,0,0:N-1]/np.sqrt(2.)*(np.abs(Phi_a0c_L)*Phase_a0c_L + np.abs(Phi_a0c_R)*Phase_a0c_R) - H * Phi_a0c
PhiPrime_a0c_TempTemp = np.flip(np.conjugate(np.flip(PhiPrime_a0c_Temp, axis=1)), axis=0)
PhiPrime_a0c = 1./2. * (PhiPrime_a0c_Temp + PhiPrime_a0c_TempTemp)

In [10]:
# Generate ab0-components of Phi

Phi_ab0_L = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Phi_ab0_R = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1))
Theta_ab0_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Theta_ab0_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1))
Phase_ab0_L = np.exp(1j*Theta_ab0_L)
Phase_ab0_R = np.exp(1j*Theta_ab0_R)

Phi_ab0_Temp = 1./np.sqrt(2.)*(np.abs(Phi_ab0_L)*Phase_ab0_L + np.abs(Phi_ab0_R)*Phase_ab0_R)
Phi_ab0_TempTemp = np.flip(np.conjugate(np.flip(Phi_ab0_Temp, axis=1)), axis=0)
Phi_ab0 = 1./2. * (Phi_ab0_Temp + Phi_ab0_TempTemp)

PhiPrime_ab0_Temp = omega[0:N-1,0:N-1,0]/np.sqrt(2.)*(np.abs(Phi_ab0_L)*Phase_ab0_L - np.abs(Phi_ab0_R)*Phase_ab0_R) - H * Phi_ab0
PhiPrime_ab0_TempTemp = np.flip(np.conjugate(np.flip(PhiPrime_ab0_Temp, axis=1)), axis=0)
PhiPrime_ab0 = 1./2. * (PhiPrime_ab0_Temp + PhiPrime_ab0_TempTemp)

In [11]:
# Generate abc-components of Phi

Phi_abc_L = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1,N-1))
Phi_abc_R = np.random.rayleigh(scale = Var**0.5, size = (N-1,N-1,N-1))
Theta_abc_L = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1,N-1))
Theta_abc_R = np.random.uniform(low = 0.0, high = 2. * np.pi, size = (N-1,N-1,N-1))
Phase_abc_L = np.exp(1j*Theta_abc_L)
Phase_abc_R = np.exp(1j*Theta_abc_R)

Phi_abc_Temp = 1./np.sqrt(2.)*(np.abs(Phi_abc_L)*Phase_abc_L + np.abs(Phi_abc_R)*Phase_abc_R)
Phi_abc_TempTemp = np.conjugate(np.flip(np.flip(np.flip(Phi_abc_Temp, axis=2), axis=1), axis=0))
Phi_abc = 1./2. * (Phi_abc_Temp + Phi_abc_TempTemp)

PhiPrime_abc_Temp = omega[0:N-1,0:N-1,0:N-1]/np.sqrt(2.)*(np.abs(Phi_abc_L)*Phase_abc_L + np.abs(Phi_abc_R)*Phase_abc_R) - H * Phi_abc
PhiPrime_abc_TempTemp = np.conjugate(np.flip(np.flip(np.flip(PhiPrime_abc_Temp, axis=2), axis=1), axis=0))
PhiPrime_abc = 1./2. * (PhiPrime_abc_Temp + PhiPrime_abc_TempTemp)

In [12]:
# Build top face of the cube.

Phi_i00 = np.concatenate((Phi_000, Phi_a00))
Phi_aj0 = np.concatenate((Phi_0b0.reshape(N-1, 1), Phi_ab0), axis=1)  # Add the column Phi0b0 to the left of Phiab0
Phi_ij0 = np.concatenate((Phi_i00.reshape(1, N), Phi_aj0), axis=0)  # Add the row Phi_i00 at the top of Phi_aj0

PhiPrime_i00 = np.concatenate((PhiPrime_000, PhiPrime_a00))
PhiPrime_aj0 = np.concatenate((PhiPrime_0b0.reshape(N-1, 1), PhiPrime_ab0), axis=1)  # Add the column Phi0b0 to the left of Phiab0
PhiPrime_ij0 = np.concatenate((PhiPrime_i00.reshape(1, N), PhiPrime_aj0), axis=0)  # Add the row Phi_i00 at the top of Phi_aj0

# Build front face of the cube (except first row).

Phi_i0c = np.concatenate((Phi_00c.reshape(1, N-1), Phi_a0c), axis=0)  # Add the row Phi00c at the top of Phia0c
PhiPrime_i0c = np.concatenate((PhiPrime_00c.reshape(1, N-1), PhiPrime_a0c), axis=0)  # Add the row Phi00c at the top of Phia0c

# Build the body of the cube

Phi_ibc = np.concatenate((Phi_0bc.reshape(1,N-1,N-1), Phi_abc), axis = 0)  # Add Phiabc to the right face Phi0bc
PhiPrime_ibc = np.concatenate((PhiPrime_0bc.reshape(1,N-1,N-1), PhiPrime_abc), axis = 0)  # Add Phiabc to the right face Phi0bc

# Build the bottom part of the cube

Phi_ijc = np.concatenate((Phi_i0c.reshape(N,1,N-1), Phi_ibc), axis = 1)
PhiPrime_ijc = np.concatenate((PhiPrime_i0c.reshape(N,1,N-1), PhiPrime_ibc), axis = 1)

# Build the entire cube

Phi_ijk = np.concatenate((Phi_ij0.reshape(N, N, 1), Phi_ijc), axis=2)  # Add the row Phi00c at the top of Phia0c
PhiPrime_ijk = np.concatenate((PhiPrime_ij0.reshape(N, N, 1), PhiPrime_ijc), axis=2)  # Add the row Phi00c at the top of Phia0c

In [13]:
# Compute inverse Fourier transform to go to real space. The result should be real.

PhiReal = np.fft.ifftn(Phi_ijk)
PhiPrimeReal = np.fft.ifftn(PhiPrime_ijk)

In [14]:
# Compute variance of PhiReal and PhiPrimeReal

print(np.std(np.imag(PhiReal)))
print(np.std(np.imag(PhiPrimeReal)))

1.8944579345060652e-21
4.45530728746865e-26


In [15]:
# Compute variance of PhiReal and PhiPrimeReal

print(np.std(PhiReal))
print(np.std(PhiPrimeReal))

1.0005976075115701e-05
2.3636140046754655e-10


In [18]:
ndim = 3
high = N
low = 0
Coords = (np.indices((high-low,)*ndim) + low).reshape(ndim, -1).T
ReshapedFR = np.real(PhiReal).reshape(N**3,1)
ReshapedFRDer = np.real(PhiPrimeReal).reshape(N**3,1)
FR = np.concatenate((Coords,ReshapedFR),axis=1)
FRDer = np.concatenate((Coords,ReshapedFRDer),axis=1)
FRDF = pd.DataFrame(FR, columns=('x','y','z','field'))
FRDFDer = pd.DataFrame(FRDer, columns=('x','y','z','field derivative'))
#FRDFreal = FRDFreal_temp
#FRDFreal_temp.head()

In [19]:
print(FRDF.head())
print(FRDFDer.head())

     x    y    z         field
0  0.0  0.0  0.0 -9.412711e-07
1  0.0  0.0  1.0 -1.274862e-05
2  0.0  0.0  2.0 -1.129158e-05
3  0.0  0.0  3.0  5.620713e-07
4  0.0  0.0  4.0  2.591732e-06
     x    y    z  field derivative
0  0.0  0.0  0.0      1.584689e-10
1  0.0  0.0  1.0     -2.003077e-10
2  0.0  0.0  2.0     -7.839084e-11
3  0.0  0.0  3.0      2.004976e-10
4  0.0  0.0  4.0      5.403886e-11


In [20]:
FRDF.to_csv('FieldRealization3D.csv', header=None, index=None)

In [21]:
FRDFDer.to_csv('FieldDerivativeRealization3D.csv', header=None, index=None)